# Deploy models using packages to Online Endpoints

In this examples, you will learn how to package model using the Model Packaging functionality to later deploy it in an online endpoint in Azure Machine Learning.

## Prerequisites

Ensure you have the latest version of `azure-ai-ml`:

```bash
%pip install -U azure-ai-ml
```

## 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

### 1.1. Import the required libraries

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    AzureMLOnlineInferencingServer,
    ModelPackage,
    CodeConfiguration,
    BaseEnvironment,
    ModelConfiguration,
)
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Environment,
    Model,
)

### 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
subscription_id = "<subscription>"
resource_group = "<resource-group>"
workspace = "<workspace>"

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

If you are running on AzureML compute, you can easily:

In [ ]:
ml_client = MLClient.from_config(DefaultAzureCredential())

## 2. Register the model

The model we will deploy is a Scikit-Learn model which is included in this repository in the folder `model`. We are going to register it:

In [ ]:
model_name = "sklearn-regression"
model_path = "model"

model = ml_client.models.create_or_update(Model(name=model_name, path=model_path))

Get the model:

In [ ]:
model = ml_client.models.get(name=model_name, label="latest")

## 3. Create the endpoint

Let's create an endpoint to host the deployment of the model.

In [ ]:
endpoint_name = "hello-packages"

Endpoint names should be unique so we will append a random string at the end to ensure that:

In [ ]:
import random
import string

# Creating a unique endpoint name by including a random suffix
allowed_chars = string.ascii_lowercase + string.digits
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))
endpoint_name = f"{endpoint_name}-{endpoint_suffix}"

print(f"Endpoint name: {endpoint_name}")

Let's create the endpoint:

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## 4. Deploy with packages

### 4.1 The base environment

Packages uses a base environment to construct the package. The base environment contains all the software dependencies required for the model to run. Notice that, unlike when deploying models to Online Endpoints, this environment doesn't need any specific package for Online Endpoints. Only include the packages required for your model to work.

In [ ]:
base_environment = ml_client.environments.create_or_update(
    Environment(
        name=f"{model_name}-env",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu22.04",
        conda_file="environment/conda.yml",
    )
)

### 4.2 Package the model

In our case, we will deploy to an online endpoint using the AzureML inferencing server, which is the default one. Since our model is not MLflow, we need to indicate a code configuration section.

In [ ]:
pakage_config = ModelPackage(
    target_environment="sklearn-regression-online-pkg",
    base_environment_source=BaseEnvironment(
        type="asset",
        resource_id=f"azureml:{base_environment.name}:{base_environment.version}",
    ),
    inferencing_server=AzureMLOnlineInferencingServer(
        code_configuration=CodeConfiguration(code="src", scoring_script="score.py")
    ),
)

Let's start the package operation:

In [ ]:
model_package = ml_client.models.package(model_name, model.version, pakage_config)

The result of the package operation is an environment:

In [ ]:
model_package

### 4.3 Deploy the package

Now, we can deploy this package in an Online Endpoint. In this case, we will create a different deployment under the same endpoint so you can compare the two of them:

In [ ]:
deployment_package = ManagedOnlineDeployment(
    name="with-package",
    endpoint_name=endpoint_name,
    environment=model_package,
    instance_count=1,
)

> Tip: Notice how model or scoring script are not being indicated in this example. All of them are part of the package.

Let's create the deployment:

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment_package).result()

### 4.4 Test the deployment

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="with-package",
    request_file="sample-request.json",
)

The method should return the prediction for two instances of data.

## 5. Deploying with packages directly from the deployment

If you don't need to configure how the package is performed, you can quickly take advantage of the packaging functionality by indicating Online Endpoints to package before performing the deployment.

To do so, indicate the argument `with_package=True`.

In [ ]:
deployment_package = ManagedOnlineDeployment(
    name="with-package-inline",
    endpoint_name=endpoint_name,
    environment=base_environment,
    model=model.id,
    code_configuration=CodeConfiguration(
            code='src', 
            scoring_script='score.py'
        )
    instance_count=1,
    with_package=True,
)

Let's create the deployment:

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment_package).result()

## 6. Deploy the model outside AzureML

Packages also allow you to generate a serving image that can be used to get the model deployed to whatever target is needed. Since AzureML is able to do some optimizations when packaging models, like for instance to mount the location where the model is stored rather than to copy the entire file inside of the image, you need to indicate to AzureML that you intend to take the image outside of the workspace.

Let's see how you can configure the package to do so:

### 5.1 Package the model

We are going to package the model again, but this time, we are going to copy the resources inside of the image so we can take it outside of AzureML.

In [ ]:
pakage_config = ModelPackage(
    target_environment="sklearn-regression-online-pkg",
    base_environment_source=BaseEnvironment(
        type="asset",
        resource_id=f"azureml:{base_environment.name}:{base_environment.version}",
    ),
    inferencing_server=AzureMLOnlineInferencingServer(
        code_configuration=CodeConfiguration(code="src", scoring_script="score.py")
    ),
    model_configuration=ModelConfiguration(mode="copy"),
)

Let's start the package operation:

In [ ]:
model_package = ml_client.models.package(model_name, model.version, pakage_config)

## 6. Clean un resources

Once done, delete the associated resources from the workspace:

In [ ]:
ml_client.online_endpoints.begin_delete(endpoint.name).result()